### admin에서 QA List를 조회하고, 추가하기 위해 DB를 만드는 과정
* 최초 한 번만 실행하면 됨(제가 실행할거니까 따로 실행 안하셔도 됨)


#### db의 종류는 2개
* vector 저장을 위한 chroma db / admin 페이지에 기존에 존재하는 qa 목록을 보여주기 위한 기본 sqlite3 => 동시에 사용
* 두 db는 동일한 qa를 담고 있음
* admin에서 보여주는 목록은 sqlite3의 내용이고, csv파일로 새로운 qa를 입력할 때 유사도 검사를 진행 한 후 통과한 내용들에 대하여 chroma db와 sqlite3 모두에 저장

1. 환경 준비

In [1]:
# 장고 설정 로드, 모델 사용
import pandas as pd
import sqlite3
import numpy as np


from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
data = pd.read_csv('faq_list.csv', encoding='utf-8')
df = pd.DataFrame(data)
df.rename(columns={'구분':'type', 'QA':'qa'}, inplace = True)

In [3]:
df

,id,type,qa
0,1,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\n\nKT 에이블스쿨은 정규 4...
1,2,모집/선발,35세 이상은 지원할 수 없나요?\n\n본 교육 과정은 34세 이하를 대상으로 하는...
2,3,모집/선발,미취업자의 기준이 뭔가요?\n\n미취업자의 기준은 아래와 같습니다.1) 기간의 정함...
3,4,모집/선발,"직장인도 지원할 수 있나요?\n\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육..."
4,5,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n\n고용보험에 가입이 되어 있는 경우...
...,...,...,...
63,64,기타,AICE (기존 AIFB 자격증이 있어야 유리한가요?\n\n네. AICE (기존 A...
64,65,기타,AICE 시험관련 문의는 어디에 해야하나요?\n\nhelp@aice.study 으로...
65,66,기타,국민취업지원제도를 신청하고 싶습니다.\n\n국민취업지원제도 신청과 관련 사항은 거주...
66,67,기타,국민취업지원제도를 안해도 교육 참여되나요?\n\n국민취업제도를 신청하지 않아도 KT...


2. sqlite에 저장

In [4]:
# DB 연결
path = './db.sqlite3'
conn = sqlite3.connect(path)


# 테이블 생성
cursor = conn.cursor()

cursor.execute('''
               CREATE TABLE IF NOT EXISTS QA_LIST(
                   id INTEGER PRIMARY KEY,
                   type CHAR NOT NULL,
                   qa CHAR NOT NULL
               )
               ''')

df.to_sql('QA_LIST', conn, if_exists='append', index=False)

conn.close()

3. vector db

1) chroma db 경로 지정

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./database",
                  embedding_function=embeddings)

In [6]:
database.get()

{'ids': ['2f72e7c3-bf2d-4dd6-9461-8f2d7c9ace05',
  '3400ec72-8e3a-49ec-a1a2-3684c0ad1d8a',
  '7fa1f098-2ec6-47fb-9a04-735b1dec71c5',
  'a808d7df-63f4-4f93-b3f6-c9defea2ae9a',
  'adc0120a-bdd0-4b21-8534-46d96bb50bb5',
  'b633a3fc-d5c1-4134-b154-743befd43c2c',
  'bce64298-103e-46e7-9929-b3b10ad524f0',
  'ee557512-244f-4e96-b912-fc254e234622',
  'f57e1c48-3389-40df-8ef5-790c5bb39526',
  'fdb2f340-1e9c-4e42-9d32-be6c574ab676'],
 'embeddings': None,
 'metadatas': [None, None, None, None, None, None, None, None, None, None],
 'documents': ['프리랜서도 지원할 수 있나요?\n고용보험에 가입하지 않은 경우 미취업자로 보지만, 프리랜서 계약의 경우 개인별 상황이 다를 수 있어서 관할 고용센터에 문의하여 확인하시기 바랍니다.\n※ 관할 고용센터 :\n수도권: 성남고용센터\n충남/충북: 대전고용센터\n대구/경북: 대구고용센터\n전남/전북: 광주고용센터\n부산/경남: 부산고용센터\n- 고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메인 하단 지역별 기관검색에서 확인 가능',
  '최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.',
  '사업자도 지원할 수 있나요?\n사업자는 자영업자로 분류됩니다. 자영업자의 경우 관

In [22]:
## 기존 데이터 삭제
d = pd.DataFrame(database.get())
d = d['ids'].tolist()
database.delete(ids=d)

Delete of nonexisting embedding ID: 1bdd09b4-4ffd-4c34-801f-34128e914cb3


Delete of nonexisting embedding ID: 1fc47b8e-d4d8-4eaf-acc4-5a5b2262e8c9
Delete of nonexisting embedding ID: 4723c87e-16bd-4ad3-97ed-e65b02a53878
Delete of nonexisting embedding ID: 9c69a64f-0e1d-4939-8e66-a6f9419fbf8d


In [23]:
database.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

2. 데이터 삽입

In [18]:
data

,id,구분,QA
0,1,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\n\nKT 에이블스쿨은 정규 4...
1,2,모집/선발,35세 이상은 지원할 수 없나요?\n\n본 교육 과정은 34세 이하를 대상으로 하는...
2,3,모집/선발,미취업자의 기준이 뭔가요?\n\n미취업자의 기준은 아래와 같습니다.1) 기간의 정함...
3,4,모집/선발,"직장인도 지원할 수 있나요?\n\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육..."
4,5,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n\n고용보험에 가입이 되어 있는 경우...
...,...,...,...
63,64,기타,AICE (기존 AIFB 자격증이 있어야 유리한가요?\n\n네. AICE (기존 A...
64,65,기타,AICE 시험관련 문의는 어디에 해야하나요?\n\nhelp@aice.study 으로...
65,66,기타,국민취업지원제도를 신청하고 싶습니다.\n\n국민취업지원제도 신청과 관련 사항은 거주...
66,67,기타,국민취업지원제도를 안해도 교육 참여되나요?\n\n국민취업제도를 신청하지 않아도 KT...


In [19]:
text_list = data['QA'].tolist()
documents = [Document(page_content=text) for text in text_list]

In [24]:
database.add_documents(documents)

['9e7856e2-93bd-4e41-bd58-2c316a831a0b',
 '95718036-d597-4e83-8496-3e63a6b71633',
 '4b55bcae-6c83-4547-bfff-736994dc75e0',
 '071ff576-aa7c-4dd0-8e78-5ce0a57e2601',
 'a57c54f5-dcbb-4882-b20a-5d8c628a6ce7',
 'dcec73f7-80c5-4494-9531-63462e1934ca',
 '30fc11f7-3b5e-4a1c-9716-ee956aa1d8a3',
 '6be1346c-cb0a-44c8-b600-1e9761e63a08',
 '547fa14e-0761-4a0b-992e-db7a71c6d780',
 '8f19ba9e-91b1-4981-bf64-be6dca0eb376',
 '1e2fe479-4b4a-4dfd-9515-cbb40ea8c430',
 '657f1f57-c401-48ab-8747-f836f3a5139d',
 'e453f21a-379e-43e2-b0e6-dc680bd13b93',
 '4aed0d34-b363-411a-9375-b95276335705',
 '35b0487b-f863-4a07-9a0a-abcfa4c6fa4b',
 '3b803821-6262-4d9f-803d-c18440b75ef0',
 'a5d3a7bd-7fbc-4a91-b2ba-221ffa23afc0',
 'a291f284-261e-4e07-82d6-e6e1648c9e21',
 'd960da7a-cbfe-4a79-8b4b-16a052255591',
 'cccc981f-daf4-4c92-a43a-7c6099de6425',
 '4a443711-3254-401d-a1cc-bfdc4d601ec3',
 'c1fdd62d-05cf-49de-90f2-64178caa3e55',
 '9d7a9b89-2d39-4e11-a9de-c557a2c78d34',
 'c9879bde-0298-4a63-8f01-d64c5edde78d',
 '7690898b-138a-

In [25]:
df = pd.DataFrame(database.get())
df

,ids,embeddings,metadatas,documents,uris,data
0,0268fd97-a571-491f-9af4-5241587d3515,None,None,대면 교육도 진행하나요?\n\n교육 과정 중 STEP1과 STEP2는 대부분 비대면...,None,None
1,071ff576-aa7c-4dd0-8e78-5ce0a57e2601,None,None,"직장인도 지원할 수 있나요?\n\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육...",None,None
2,1e2fe479-4b4a-4dfd-9515-cbb40ea8c430,None,None,K-DT가 아닌 국비과정 참여 중인데 지원할 수 있나요?\n\n기존에 수강하신 과정...,None,None
3,278465bd-1fef-472f-be42-023027bce6e6,None,None,대면 교육 시 숙소가 제공 되나요?\n\n숙소는 제공하지 않습니다.,None,None
4,290e204f-adb0-457e-ac48-f939d146f22f,None,None,훈련장려금은 모두가 다 받을 수 있나요?\n\nKT 에이블스쿨 과정 수강생이라면 기...,None,None
...,...,...,...,...,...,...
63,debaf884-2a84-4947-b05a-487af37add51,None,None,자영업자 고용보험 가입 시에는 훈련장려금 못받나요?\n\n자영업자 중 고용보험 임의...,None,None
64,e453f21a-379e-43e2-b0e6-dc680bd13b93,None,None,합격 후 과정을 포기하면 재지원이 불가능한가요?\n\n교육 등록신청을 하지 않거나 ...,None,None
65,ebce656b-dc80-4d10-94fb-cccae4c21848,None,None,"훈련장려금은 얼마가 어떻게 지급되나요?\n\n훈련장려금은 월 최대 116,000원이...",None,None
66,ef04aaf6-75eb-402b-976c-210cdfcb50de,None,None,"교육을 중도 포기할 경우 패널티가 있나요?\n\n교육 시작 후 7일(주말, 공휴일 ...",None,None
